<a href="https://colab.research.google.com/github/biniyam-mulugeta/test_co_2/blob/main/c01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np 
#from keras.models import Sequential
#from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.utils.all_utils import to_categorical
from keras.layers.merge import concatenate
from keras.applications import mobilenet_v2,densenet
from keras.models import Model
from sklearn import preprocessing
import cv2
from pathlib import Path
import os
import glob
import matplotlib.pyplot as plt
#import seaborn as sns
#import zipfile
%matplotlib inline

In [ ]:
from sklearn.decomposition import PCA
#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
from keras.layers import Flatten

In [ ]:
def k_means_segmentation(img):
  twoDimage = img.reshape((-1,3))
  twoDimage = np.float32(twoDimage)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 2
  attempts=1
  ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
  center = np.uint8(center)
  res = center[label.flatten()]
  result_image = res.reshape((img.shape))
  return result_image

In [ ]:
def color_mask_segmentation(img):
  hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  light_blue = (90, 70, 50)
  #dark_blue = (128, 255, 255)
  # You can use the following values for green
  light_green = (40, 40, 40)
  # dark_greek = (70, 255, 255)
  mask = cv2.inRange(hsv_img, light_blue, dark_blue)
  result = cv2.bitwise_and(img, img, mask=mask)
  return result

In [ ]:
SIZE = 224
train_image = []
train_label = []
for dir_path in glob.glob("/content/drive/My Drive/train_cro/*"):
    label = dir_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        print(img_path)
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        #img = color_mask_segmentation(img)
        train_image.append(img)
        train_label.append(label)
train_image = np.array(train_image)
train_label = np.array(train_label)

val_image = []
val_label = []
for dir_path_v in glob.glob("/content/drive/My Drive/validation_cro/*"):
    label_v = dir_path_v.split("\\")[-1]
    #print(label_v)
    for img_path_v in glob.glob(os.path.join(dir_path_v,"*.jpg")):
        #print(img_path_v)
        img = cv2.imread(img_path_v,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        #img = color_mask_segmentation(img)
        val_image.append(img)
        val_label.append(label_v)
val_image = np.array(val_image)
val_label = np.array(val_label)

In [ ]:
from skimage import io
img = train_image[1]
print(img.shape)
#img = cv2.imread(val_image[0])
io.imshow(img)
plt.show()

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(train_label)
train_labe_encoded = le.transform(train_label)
le.fit(val_label)
val_label_encoded = le.transform(val_label)

In [ ]:
x_train,y_train,x_test,y_test = train_image,train_labe_encoded,val_image,val_label_encoded

In [ ]:
x_train,x_test = x_train/255.0,x_test/255.0

In [ ]:
x_train.shape

In [ ]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
model_1 = mobilenet_v2.MobileNetV2(include_top=False,weights=None,input_shape=(224,224,3))

In [ ]:
#model_1.summary()

In [ ]:
model_2 = densenet.DenseNet121(include_top=False,weights=None,input_shape=(224,224,3))

In [ ]:
#model = vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [ ]:
#for layer in model.layers:
#  layer.trainable = False

In [ ]:
#out = model.get_layer(index = -1).output
#out = Dropout(0.5)(out)
#out = Flatten()(out)
#out = Dense(3,activation='softmax')(out)

In [ ]:
#vgg = Model(inputs = model.input,outputs = out)

In [ ]:
#vgg.summary()

In [ ]:
#vgg.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#history= vgg.fit(x_train,y_train_one_hot,epochs=50,validation_data=(x_test,y_test_one_hot))

In [ ]:
x1 = model_1.get_layer(index = -1).output
x2 = model_2.get_layer(index = -1).output
x1 = Flatten()(x1)
x2 = Flatten()(x2)
out = concatenate([x1,x2])

In [ ]:
model = Model(inputs = ([model_1.input,model_2.input]),outputs = out)
#model.summary()

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

In [ ]:
print("Feature extraction ...")
prediction = np.array(model.predict([x_train,x_train]))
#Xtrain = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [ ]:
Xtrain = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [ ]:
prediction = np.array(model.predict([x_test,x_test]))

In [ ]:
Xtest = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', Xtrain.shape)
print('\tFeatures testing shape: ', Xtest.shape)

In [ ]:
print("Classification with Linear SVM ...")
svm = SVC(kernel='linear')
svm.fit(Xtrain, np.ravel(y_train, order='C'))
result = svm.predict(Xtest)

acc = accuracy_score(result, np.ravel(y_test, order='C'))
print("\tAccuracy Linear SVM: %0.4f" % acc)

### using random forest

In [ ]:
rf = RandomForestClassifier(n_estimators=10, random_state=30)
rf.fit(([Xtrain,Xtrain]),y_train_one_hot)
result = svm.predict(Xtest)

acc = accuracy_score(result, np.ravel(y_test_one_hot, order='C'))
print("\tAccuracy Linear SVM: %0.4f" % acc)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label= 'Validation Accuracy')
#plt.plot(epochs, loss,'', label = 'loss')
#plt.plot(epochs, val_loss,'', label = 'val loss')
plt.title('Training and Validation accuray')
plt.legend(loc=0)
plt.figure()
#show the graph
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

#plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
#plt.plot(epochs, val_acc, 'b', label= 'Validation Accuracy')
plt.plot(epochs, loss,'', label = 'loss')
plt.plot(epochs, val_loss,'', label = 'val loss')
plt.title('Training and Validation accuray')
plt.legend(loc=0)
plt.figure()
#show the graph
plt.show()

In [ ]:
#model =  densenet.DenseNet121()
model.summary()

In [ ]:
#x = Flatten()(model.get_layer(index=-2).output)

In [ ]:
model = Model(inputs = [model.input,model.input],outputs = model.get_layer(index=-4).output)

In [ ]:
model.summary()

In [ ]:
print("Feature extraction ...")
prediction = np.array(model.predict(x_train))
Xtrain = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [ ]:
prediction = np.array(model.predict(x_test))
Xtest = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', Xtrain.shape)
print('\tFeatures testing shape: ', Xtest.shape)

In [ ]:
print("Classification with Linear SVM ...")
svm = SVC(kernel='linear')
svm.fit(Xtrain, np.ravel(y_train, order='C'))
result = svm.predict(Xtest)

acc = accuracy_score(result, np.ravel(y_test, order='C'))
print("\tAccuracy Linear SVM: %0.4f" % acc)

In [ ]:
#from skle
from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from keras.applications import vgg16
import numpy as np

# Some random images, labels and target label
images = np.random.rand(10, 64, 64, 3)
labels = np.random.randint(0, 1, size=(10, 4))
target = np.random.randint(0, 1, size=(10, 1))

# Extract VGG16 features for the images
image_input = Input((64, 64, 3))
model = vgg16.VGG16(include_top=False, weights='imagenet')
features = model.predict(images)
features = np.reshape(features, (10, -1))  # 2048 features per image 

# Two input layers: one for the image features, one for additional labels
feature_input = Input((2048,), name='feature_input')
label_input = Input((4, ), name='label_input')

# Concatenate the features
concatenate_layer = Concatenate(name='concatenation')([feature_input, label_input]) 
dense = Dense(16)(concatenate_layer)
output = Dense(1, name='output_layer', activation='sigmoid')(dense)

# To define the model, pass list of input layers
model = Model(inputs=[feature_input, label_input], outputs=output)
model.compile(optimizer='sgd', loss='binary_crossentropy')

# To fit the model, pass a list of inputs arrays
model.fit(x=[features, labels], y=target)

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)